In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
# Load the dataset
lines = open('./dorothea_train.data').read().splitlines() ## Train

In [3]:
# Declare Empty X matrix
tmp=np.zeros(shape=(800,100000))
X=pd.DataFrame(tmp)

# Now fill X matrix
for i in range(0,len(lines)):
    row=lines[i]
    row=row.split()
    row = list(map(int, row)) ## convert string indexes as int indexes
    row=np.array(row) ## Convert to numpy array to apply arithmetic operation
    row=row-1 ## convert index from 1 to 100000 to 0 to 99999
    X.ix[i][row]=1

In [4]:
#Calculated Centred Matrix A
mean=X.mean()
A=X-mean
A=A.as_matrix()   ## Convert to numpy from DF
A=np.transpose(A) ## A is d*n
At=np.transpose(A)
X=X.as_matrix()
X=np.transpose(X)   ## X is now d*n
# Finally A is centred data, X is normal data , both being d*n

In [142]:
K=np.dot(At,A)

In [143]:
tmp=np.linalg.eig(K)
eign_values=tmp[0]
eign_vector=tmp[1]

In [144]:
## Choose 'k' biggest eign vectors
num_of_comp=800 #all DUMMY
indexes = eign_values.argsort()[::-1]
eign_vector = eign_vector[:, indexes[0:num_of_comp]]

In [145]:
W=np.dot(A,eign_vector)

In [146]:
rootlmda=np.sqrt(eign_values)
W=W/rootlmda

In [147]:
# ## Reducing to 100 
# W=np.transpose(W)
features=800
W=W[:,:features]


In [148]:
reducedA=np.dot(np.transpose(W),X)

In [149]:
## Bring back to normal representation
reducedA=np.transpose(reducedA)
## Now reducedA is datapoints (n*d) in reduced Dimension

In [150]:
########################################## PCA IS COMPLETE ###################################

In [151]:
############## READ TEST DATA ##########################
lines_test=open('./dorothea_valid.data').read().splitlines() ## Test
# Declare Empty X matrix
tmp=np.zeros(shape=(350,100000))
X_test=pd.DataFrame(tmp)

# Now fill X matrix
for i in range(0,len(lines_test)):
    row=lines_test[i]
    row=row.split()
    row = list(map(int, row)) ## convert string indexes as int indexes
    row=np.array(row) ## Convert to numpy array to apply arithmetic operation
    row=row-1 ## convert index from 1 to 100000 to 0 to 99999
    X_test.ix[i][row]=1

In [152]:
X_test=X_test.as_matrix()
X_test=np.transpose(X_test)   ## X_test is now d*n
reduced_X_test=np.dot(np.transpose(W),X_test)

In [153]:
## Bring back to normal representation
reduced_X_test=np.transpose(reduced_X_test)
## Now reducedA is datapoints (n*d) in reduced Dimension

In [154]:
## Divide data in train and test
train=reducedA
test=reduced_X_test

In [155]:
## Now start Training 
#Read lables
lines = open('./dorothea_train.labels').read().splitlines()
lines_test = open('./dorothea_valid.labels').read().splitlines()
train_lables=lines
test_lables=lines_test    

In [156]:
## Split train data according to classes
#Add class lables to data
df=pd.DataFrame(train)
tmp=pd.Series(train_lables)
df.loc[:,features+1]=tmp
#Now split as per lables
df0=df[df[features+1]=='-1'] 
df1=df[df[features+1]=='1']  

## Remove class lables now
del df0[features+1] ## DF containing class -1 
del df1[features+1] ## DF containing class +1
## convert to numpy arrays
class0=df0.as_matrix()
class1=df1.as_matrix()

In [157]:
## Now find mean and covariance
u1=class0.mean(0)
u2=class1.mean(0)
e1=np.cov(class0,rowvar=False)
e2=np.cov(class1,rowvar=False) 
## Also calculate posterior prob
class0_pos=len(class0)/len(train)
class1_pos=len(class1)/len(train)

In [158]:
co1=e1.diagonal()
co2=e2.diagonal()
co1=np.dot(np.transpose(co1),co1)
co2=np.dot(np.transpose(co2),co2)

In [159]:
## Calculate p(x) from Gaussian Formula
import math
count=0
for i in range (0 , len(test)):
    ## CLass 0
    sample=test[i]
    sm=0
    for j in range (0,len(sample)): 
        xminusu=(sample[j]-u1[j])
        xminusu=xminusu * xminusu
        tmp=xminusu/e1[j][j]
        tmp=tmp*(-0.5)
        e=math.exp(tmp)
        e=e/(math.sqrt(2*math.pi*e1[j][j]))
        sm=sm+e
    p0=sm*class0_pos
    
    ## Class 1
    sample=test[i]
    sm=0
    for j in range (0,len(sample)): 
        xminusu=(sample[j]-u2[j])
        xminusu=xminusu * xminusu
        tmp=xminusu/e2[j][j]
        tmp=tmp*(-0.5)
        e=math.exp(tmp)
        e=e/(math.sqrt(2*math.pi*e2[j][j]))
        sm=sm+e
    p1=sm*class1_pos
    
    
    if(p0>p1):
        result='-1'
    else : 
        result='1'
    if(result==test_lables[i]):
        count+=1
    

print(float(count)/len(test))

0.9028571428571428


In [160]:
from sklearn.naive_bayes import GaussianNB
abc=GaussianNB()
abc.fit(train,train_lables)
print(abc.score(test,test_lables))

0.902857142857
